In [ ]:
import os
from src import fetchReview
from src.data_extraction_pipeline import ReviewExtractionPipeline

In [ ]:
app_info = {
    "google_play_store_info":'com.phonepe.app',
    "apple_play_store_info" :"https://apps.apple.com/in/app/phonepe-secure-payments-app/id1170055821"
}
rep = ReviewExtractionPipeline(company_name="PhonePe",app_info=app_info)


In [ ]:
rep.write()

Class objects create for review extraction


2024-02-15 15:33:58,891 [INFO] Base - Initialised: AppStore('in', 'phonepe-secure-payments-app', 1170055821)
2024-02-15 15:33:58,891 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/in/app/phonepe-secure-payments-app/id1170055821
2024-02-15 15:34:03,912 [INFO] Base - [id:1170055821] Fetched 240 reviews (240 fetched in total)
2024-02-15 15:34:09,505 [INFO] Base - [id:1170055821] Fetched 540 reviews (540 fetched in total)
2024-02-15 15:34:14,956 [INFO] Base - [id:1170055821] Fetched 820 reviews (820 fetched in total)
2024-02-15 15:34:20,518 [INFO] Base - [id:1170055821] Fetched 1120 reviews (1120 fetched in total)
2024-02-15 15:34:26,219 [INFO] Base - [id:1170055821] Fetched 1420 reviews (1420 fetched in total)
2024-02-15 15:34:31,889 [INFO] Base - [id:1170055821] Fetched 1720 reviews (1720 fetched in total)


In [ ]:
from src.fetchReview import GoogleAppReviewExtractionPipeline,AppleAppReviewExtractionPipeline


class ReviewExtractionPipeline():
    def __init__(self,company_name,app_info):
        self.company_name = company_name
        self.google_app_id = app_info["google_play_store_info"]
        self.apple_app_id = app_info["apple_play_store_info"]
        
        # self.google_app_id = self.google_store_info.app_id
        # self.apple_app_id = self.apple_store_info.app_url
        dirname = os.path.dirname(__file__)
        self.output_path = os.path.join(dirname, "data",company_name)

    def extract_review(self):
        google_reviews = GoogleAppReviewExtractionPipeline(self.google_app_id)
        apple_reviews = AppleAppReviewExtractionPipeline(self.apple_app_id)
        return google_reviews, apple_reviews
    
    def combine_reviews(self):

        # extracting review
        google_df, apple_df = self.extract_review()
        
        # app review - rearranging columns
        apple_df["content"] = apple_df[["title","review"]].fillna("").agg('. '.join, axis=1)
        apple_df.loc[:,"thumbsUpCount"] = 1
        apple_df = apple_df[["date","rating","thumbsUpCount","content"]]
        apple_df.columns = ["date","rating","thumbsUpCount","review"]
        apple_df.loc[:,"source"] = "Apple Play Store"

        # google review - rearranging columns
        google_df = google_df[["at","score","thumbsUpCount","content"]]
        google_df.columns = ["date","rating","thumbsUpCount","review"]
        google_df.loc[:,"source"] = "Google Play Store"
        
        # combining reviews
        all_reviews_df =  pd.concat([google_df,apple_df],axis=0)
        
        return all_reviews_df
    def write(self):
        df = self.combine_reviews()
        df.to_csv(self.output_path,index=False)

        
        
if __name__ =='__main__':
    app_info = {
        "google_play_store_info":'com.phonepe.app',
        "apple_play_store_info" :"https://apps.apple.com/in/app/phonepe-secure-payments-app/id1170055821"
    }
    rep = ReviewExtractionPipeline(company_name="PhonePe",app_info=app_info)
    rep.write()


    


NameError: name '__file__' is not defined